This notebook will be used to load and process all the data that will be needed for the visualization part. This notebook is divided into three parts:
1. Internally Displaced Persons: Data obtained through (https://data.humdata.org/dataset/idmc-internally-displaced-persons-idps-new-displacement-associated-with-disasters) with info about the number of people internally displaced associated to natural disasters. Loading and cleaning the df.
2. Average temperature 1991-2016: Data about the earth surface temperature (https://climate.nasa.gov/vital-signs/global-temperature/). Loading and cleaning the df.
3. Merging both dfs

# 1. Internally Displaced Persons


In [1]:
import pandas as pd
import pycountry_convert as pc

#### We define a function that takes the iso3 standard of a country and returns the continent code to which that country belongs

In [2]:
def continent(country_code):
    try:
        return pc.country_alpha2_to_continent_code(pc.country_alpha3_to_country_alpha2(country_code))
    except:
        return "no_continent"

#### Loading data

In [3]:
idp=pd.read_csv("input/disaster_data.csv")

In [4]:
idp.head()

,ISO3,Name,Year,Start Date,Event Name,Hazard Category,Hazard Type,New Displacements
0,#country+code,#country+name,#date+year,#date+start,#description,#crisis+category,#crisis+type,#affected+idps+ind+newdisp+disaster
1,AB9,Abyei Area,2018,2018-07-01,Abyie: Flood - 01/07/2018,Weather related,Flood,2
2,AFG,Afghanistan,2008,2008-01-01,NaN,Weather related,Extreme temperature,NaN
3,AFG,Afghanistan,2008,2008-08-01,NaN,Weather related,Flood,180
4,AFG,Afghanistan,2008,2008-04-17,NaN,Geophysical,Earthquake,3250


#### Cleaning first row

In [5]:
idp.drop(0,inplace=True)

In [6]:
idp.head()

,ISO3,Name,Year,Start Date,Event Name,Hazard Category,Hazard Type,New Displacements
1,AB9,Abyei Area,2018,2018-07-01,Abyie: Flood - 01/07/2018,Weather related,Flood,2
2,AFG,Afghanistan,2008,2008-01-01,NaN,Weather related,Extreme temperature,NaN
3,AFG,Afghanistan,2008,2008-08-01,NaN,Weather related,Flood,180
4,AFG,Afghanistan,2008,2008-04-17,NaN,Geophysical,Earthquake,3250
5,AFG,Afghanistan,2009,2009-01-01,NaN,Weather related,Flood,25185


#### Checking types

In [7]:
idp.dtypes

ISO3                 object
Name                 object
Year                 object
Start Date           object
Event Name           object
Hazard Category      object
Hazard Type          object
New Displacements    object
dtype: object

Changing types for `Year` `Start Date` `New Displacements`

In [8]:
idp["Start Date"]=pd.to_datetime(idp["Start Date"]).dt.month
idp["Year"]=idp["Year"].astype(int)

We need to fill the NaNs with zeros. Those NaNs means that althougth an event took place, there were no affected persons

In [9]:
idp["New Displacements"].fillna(0,inplace=True)

In [10]:
idp["New Displacements"]=idp["New Displacements"].astype(int)

Lets fill the NaNs in the `Event Name` column with '-' so that we know that there is no additional info related to that event

In [11]:
idp["Event Name"].fillna("-",inplace=True)

In [12]:
idp.dtypes

ISO3                 object
Name                 object
Year                  int64
Start Date            int64
Event Name           object
Hazard Category      object
Hazard Type          object
New Displacements     int64
dtype: object

#### Changing column names

In [13]:
idp.columns=(idp.columns
             .str.replace(" ","_")
             .str.lower()
             .str.replace("new_displacements","affected")
             .str.replace("start_date","month")
             .str.replace("name","country")
            )

In [14]:
idp.head()

,iso3,country,year,month,event_country,hazard_category,hazard_type,affected
1,AB9,Abyei Area,2018,7,Abyie: Flood - 01/07/2018,Weather related,Flood,2
2,AFG,Afghanistan,2008,1,-,Weather related,Extreme temperature,0
3,AFG,Afghanistan,2008,8,-,Weather related,Flood,180
4,AFG,Afghanistan,2008,4,-,Geophysical,Earthquake,3250
5,AFG,Afghanistan,2009,1,-,Weather related,Flood,25185


Exploring the different categories in our dataset

In [15]:
idp["hazard_type"].value_counts()

Flood                      3086
Storm                      1656
Wet mass movement           526
Wildfire                    512
Earthquake                  327
Extreme temperature          95
Dry mass movement            93
Volcanic eruption            73
Mass movement                30
Drought                      14
Volcanic activity            13
Wet Mass Movement             1
Wet Mass movement             1
Severe winter condition       1
Name: hazard_type, dtype: int64

We observe same category with different names, we proceed to streamline that

In [16]:
idp["hazard_type"]=(idp["hazard_type"]
                    .str.lower()
                    .str.replace("wet","")
                    .str.replace("dry","")
                    .str.replace("activity","eruption")
                    .str.strip()
                   )
idp["hazard_type"].value_counts()

flood                      3086
storm                      1656
mass movement               651
wildfire                    512
earthquake                  327
extreme temperature          95
volcanic eruption            86
drought                      14
severe winter condition       1
Name: hazard_type, dtype: int64

In [17]:
idp["hazard_category"].value_counts()

Weather related    5914
Geophysical         514
Name: hazard_category, dtype: int64

#### Checking the name of the countries

In [18]:
idp["country"].unique()

array(['Abyei Area', 'Afghanistan', 'Angola', 'Anguilla', 'Albania',
       'United Arab Emirates', 'Argentina', 'American Samoa',
       'Antigua and Barbuda', 'Australia', 'Austria', 'Azerbaijan',
       'Burundi', 'Belgium', 'Benin', 'Burkina Faso', 'Bangladesh',
       'Bulgaria', 'Bahamas, The', 'Bahamas', 'Bosnia and Herzegovina',
       'Belize', 'Bolivia', 'Brazil', 'Barbados', 'Brunei Darussalam',
       'Brunei', 'Bhutan', 'Botswana', 'Central African Republic',
       'Canada', 'Switzerland', 'Chile', 'China', "Côte d'Ivoire",
       'Cameroon', 'Congo, Dem. Rep.', 'Dem. Rep. Congo', 'Ukraine',
       'Colombia', 'Comoros', 'Cabo Verde', 'Costa Rica', 'Cuba',
       'Cayman Islands', 'Cyprus', 'Czech Republic', 'Germany',
       'Djibouti', 'Dominica', 'Dominican Republic', 'Algeria', 'Ecuador',
       'Egypt, Arab Rep.', 'Egypt', 'Eritrea', 'Spain', 'Ethiopia',
       'Finland', 'Fiji', 'France', 'Micronesia, Fed. Sts.', 'Gabon',
       'United Kingdom', 'Georgia', 'Ghana',

#### We observe that there are wrong country names, or not streamlined. We that now

In [19]:
to_change_country={'Bahamas, The':'Bahamas',
                   'Brunei Darussalam':'Brunei',
                   'Congo, Dem. Rep.':'Dem. Rep. Congo',
                   'Egypt, Arab Rep.':'Egypt',
                   'Micronesia, Fed. Sts.':'Micronesia',
                   'Gambia, The':'Gambia',
                   'Hong Kong SAR, China':'Hong Kong',
                   'Hong Kong, China':'Hong Kong',
                   'Iran, Islamic Rep.':'Iran',
                   'Kyrgyz Republic':'Kyrgyzstan',
                   'Korea, Rep.':'South Korea',
                   'St. Martin (French part)':'St. Martin',
                   'Sint Maarten (Dutch part)':'St. Martin',
                   'Macedonia, FYR':'Macedonia',
                   'Korea, Dem. Rep.':'North Korea',
                   "Dem. People's Rep. Korea":"North Korea",
                   'West Bank and Gaza':'Palestine',
                   'Russian Federation':'Russia',
                   'Slovak Republic':'Slovakia',
                   'Syrian Arab Republic':'Syria',
                   'Taiwan, China':"Taiwan",
                   'Venezuela, RB':'Venezuela',
                   'Viet Nam':'Vietnam',
                   'British Virgin Islands':'Virgin Islands',
                   'Yemen, Rep.':'Yemen'
                  }

In [20]:
idp.replace({"country": to_change_country},inplace=True)

#### Check that it worked

In [21]:
idp["country"].unique()

array(['Abyei Area', 'Afghanistan', 'Angola', 'Anguilla', 'Albania',
       'United Arab Emirates', 'Argentina', 'American Samoa',
       'Antigua and Barbuda', 'Australia', 'Austria', 'Azerbaijan',
       'Burundi', 'Belgium', 'Benin', 'Burkina Faso', 'Bangladesh',
       'Bulgaria', 'Bahamas', 'Bosnia and Herzegovina', 'Belize',
       'Bolivia', 'Brazil', 'Barbados', 'Brunei', 'Bhutan', 'Botswana',
       'Central African Republic', 'Canada', 'Switzerland', 'Chile',
       'China', "Côte d'Ivoire", 'Cameroon', 'Dem. Rep. Congo', 'Ukraine',
       'Colombia', 'Comoros', 'Cabo Verde', 'Costa Rica', 'Cuba',
       'Cayman Islands', 'Cyprus', 'Czech Republic', 'Germany',
       'Djibouti', 'Dominica', 'Dominican Republic', 'Algeria', 'Ecuador',
       'Egypt', 'Eritrea', 'Spain', 'Ethiopia', 'Finland', 'Fiji',
       'France', 'Micronesia', 'Gabon', 'United Kingdom', 'Georgia',
       'Ghana', 'Guinea', 'Gambia', 'Guinea-Bissau', 'Greece', 'Grenada',
       'Greenland', 'Guatemala', 'Gu

### Getting the continents from the country code

In [22]:
idp=idp.assign(continent=idp["iso3"].apply(lambda x: continent(x)))

In [23]:
idp.head()

,iso3,country,year,month,event_country,hazard_category,hazard_type,affected,continent
1,AB9,Abyei Area,2018,7,Abyie: Flood - 01/07/2018,Weather related,flood,2,no_continent
2,AFG,Afghanistan,2008,1,-,Weather related,extreme temperature,0,AS
3,AFG,Afghanistan,2008,8,-,Weather related,flood,180,AS
4,AFG,Afghanistan,2008,4,-,Geophysical,earthquake,3250,AS
5,AFG,Afghanistan,2009,1,-,Weather related,flood,25185,AS


#### Now we have the dataset clean and ready to be merged

# 2. Average temperature 1991-2016

#### Loading the data

In [24]:
avg_temp=pd.read_csv("input/avg_temp.csv",sep=",")

In [25]:
avg_temp.head()

,Temperature - (Celsius),Year,Statistics,Country,ISO3
0,11.1664,1991,Jan Average,Algeria,DZA
1,14.1740,1991,Feb Average,Algeria,DZA
2,18.5296,1991,Mar Average,Algeria,DZA
3,21.4247,1991,Apr Average,Algeria,DZA
4,24.9652,1991,May Average,Algeria,DZA


#### Renaming columns

In [26]:
avg_temp.columns=(avg_temp.columns
                  .str.strip()
                  .str.lower()
                  .str.replace(" ","")
                  .str.replace("-","")
                  .str.replace("(","")
                  .str.replace(")","")
                  .str.replace("temperaturecelsius","temp")
                  .str.replace("statistics","month")
                 )

Reordering column names

In [27]:
cols=list(avg_temp.columns.values)
new_cols=[]
new_cols.append([cols[-1],cols[-2],cols[1],cols[2],cols[0]])
new_cols=new_cols[0]
new_cols

['iso3', 'country', 'year', 'month', 'temp']

In [28]:
avg_temp=avg_temp[new_cols]
avg_temp.head()

,iso3,country,year,month,temp
0,DZA,Algeria,1991,Jan Average,11.1664
1,DZA,Algeria,1991,Feb Average,14.1740
2,DZA,Algeria,1991,Mar Average,18.5296
3,DZA,Algeria,1991,Apr Average,21.4247
4,DZA,Algeria,1991,May Average,24.9652


#### Checking types

In [29]:
avg_temp.dtypes

iso3        object
country     object
year         int64
month       object
temp       float64
dtype: object

We need to get the month column just with the month number

In [30]:
avg_temp["month"]=avg_temp["month"].str.split().str[0]

In [31]:
avg_temp.head()

,iso3,country,year,month,temp
0,DZA,Algeria,1991,Jan,11.1664
1,DZA,Algeria,1991,Feb,14.1740
2,DZA,Algeria,1991,Mar,18.5296
3,DZA,Algeria,1991,Apr,21.4247
4,DZA,Algeria,1991,May,24.9652


In [32]:
avg_temp["month"]=pd.to_datetime(avg_temp["month"],format="%b").dt.month

In [33]:
avg_temp.head()

,iso3,country,year,month,temp
0,DZA,Algeria,1991,1,11.1664
1,DZA,Algeria,1991,2,14.1740
2,DZA,Algeria,1991,3,18.5296
3,DZA,Algeria,1991,4,21.4247
4,DZA,Algeria,1991,5,24.9652


#### Taking out the space before the country

In [34]:
avg_temp["country"]=avg_temp["country"].str.strip()
avg_temp["iso3"]=avg_temp["iso3"].str.strip()

#### Now the dataset is ready to be merged.

# 3. Merging idp and average temperatures dataframes

#### Having a look to both dfs

In [35]:
idp.shape

(6431, 9)

In [36]:
idp.head()

,iso3,country,year,month,event_country,hazard_category,hazard_type,affected,continent
1,AB9,Abyei Area,2018,7,Abyie: Flood - 01/07/2018,Weather related,flood,2,no_continent
2,AFG,Afghanistan,2008,1,-,Weather related,extreme temperature,0,AS
3,AFG,Afghanistan,2008,8,-,Weather related,flood,180,AS
4,AFG,Afghanistan,2008,4,-,Geophysical,earthquake,3250,AS
5,AFG,Afghanistan,2009,1,-,Weather related,flood,25185,AS


In [37]:
avg_temp.shape

(40872, 5)

In [38]:
avg_temp.head()

,iso3,country,year,month,temp
0,DZA,Algeria,1991,1,11.1664
1,DZA,Algeria,1991,2,14.1740
2,DZA,Algeria,1991,3,18.5296
3,DZA,Algeria,1991,4,21.4247
4,DZA,Algeria,1991,5,24.9652


#### Merging them

In [39]:
merge=idp.merge(avg_temp,how="left",on=["iso3","year","month"])
merge.head()

,iso3,country_x,year,month,event_country,hazard_category,hazard_type,affected,continent,country_y,temp
0,AB9,Abyei Area,2018,7,Abyie: Flood - 01/07/2018,Weather related,flood,2,no_continent,NaN,NaN
1,AFG,Afghanistan,2008,1,-,Weather related,extreme temperature,0,AS,NaN,NaN
2,AFG,Afghanistan,2008,8,-,Weather related,flood,180,AS,NaN,NaN
3,AFG,Afghanistan,2008,4,-,Geophysical,earthquake,3250,AS,NaN,NaN
4,AFG,Afghanistan,2009,1,-,Weather related,flood,25185,AS,NaN,NaN


**NOTE:** 
- The amount of NaNs is due to:
    1. the non existence of data for 80 of the 211 countries in the idp df
    2. the non existence of data for the years 2017 and 2018

Deleting the country_y column and renaiming country_x to just country

In [40]:
merge.drop("country_y",axis=1,inplace=True)

In [41]:
merge.columns=merge.columns.str.replace("country_x","country")

In [42]:
merge.head()

,iso3,country,year,month,event_country,hazard_category,hazard_type,affected,continent,temp
0,AB9,Abyei Area,2018,7,Abyie: Flood - 01/07/2018,Weather related,flood,2,no_continent,NaN
1,AFG,Afghanistan,2008,1,-,Weather related,extreme temperature,0,AS,NaN
2,AFG,Afghanistan,2008,8,-,Weather related,flood,180,AS,NaN
3,AFG,Afghanistan,2008,4,-,Geophysical,earthquake,3250,AS,NaN
4,AFG,Afghanistan,2009,1,-,Weather related,flood,25185,AS,NaN


Creating a column with the year and the month with a datetime type

In [43]:
merge=merge.assign(year_month=pd.to_datetime(merge["year"].astype(str)+"-"+merge["month"].astype(str)+"-01"))

Rearrenging the order

In [44]:
merge=merge[["iso3","country","continent","year","month","year_month","event_country","hazard_category","hazard_type","affected","temp"]]

In [45]:
merge.head()

,iso3,country,continent,year,month,year_month,event_country,hazard_category,hazard_type,affected,temp
0,AB9,Abyei Area,no_continent,2018,7,2018-07-01,Abyie: Flood - 01/07/2018,Weather related,flood,2,NaN
1,AFG,Afghanistan,AS,2008,1,2008-01-01,-,Weather related,extreme temperature,0,NaN
2,AFG,Afghanistan,AS,2008,8,2008-08-01,-,Weather related,flood,180,NaN
3,AFG,Afghanistan,AS,2008,4,2008-04-01,-,Geophysical,earthquake,3250,NaN
4,AFG,Afghanistan,AS,2009,1,2009-01-01,-,Weather related,flood,25185,NaN


#### Checking that the types remain correct

In [46]:
merge.dtypes

iso3                       object
country                    object
continent                  object
year                        int64
month                       int64
year_month         datetime64[ns]
event_country              object
hazard_category            object
hazard_type                object
affected                    int64
temp                      float64
dtype: object

### Exporting as a pickle file the merged df

In [47]:
merge.to_pickle("output/merged.pkl")